In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

# df = pd.read_csv('stack-overflow-data.csv')
# df = df[pd.notnull(df['tags'])]
# print(df.head(10))
# print(df['post'].apply(lambda x: len(x.split(' '))).sum())

unable to import 'smart_open.gcs', disabling that module


In [2]:
# print(df['post'].head(10))

In [3]:
import json
import pdb
import os

def get_tweet_text_from_json(file_path):
    events = []
    labels = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
            labels.append(value["label"])
        return events,labels
    
def get_tweet_text_from_json_unlabel(file_path):
    events = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
        return events
    
misinfo_train_events,misinfo_train_labels = get_tweet_text_from_json("train.json")
dev_events,dev_labels = get_tweet_text_from_json("dev.json")
test_unlabel = get_tweet_text_from_json_unlabel("test-unlabelled.json")

In [6]:
# dev_info = [i for i,x in enumerate(dev_labels) if x==0]
# print(dev_info)

In [4]:
def anti_abbrevation(text):
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"cannot", "can not ", text)
    text = re.sub(r"what\'s", "what is", text)
    text = re.sub(r"What\'s", "what is", text)
    text = re.sub(r"\'ve ", " have ", text)
    text = re.sub(r"n\'t", " not ", text)
    text = re.sub(r"i\'m", "i am ", text)
    text = re.sub(r"I\'m", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r" e mail ", " email ", text)
    text = re.sub(r" e \- mail ", " email ", text)
    text = re.sub(r" e\-mail ", " email ", text)
    
    text = re.sub(r"what're", "can not", text)
    text = re.sub(r"cannot", "can not ", text)
    text = re.sub(r"what\'s", "what is", text)
    text = re.sub(r"What\'s", "what is", text)
    text = re.sub(r"\'ve ", " have ", text)
    text = re.sub(r"n\'t", " not ", text)
    text = re.sub(r"i\'m", "i am ", text)
    text = re.sub(r"I\'m", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r" e mail ", " email ", text)
    text = re.sub(r" e \- mail ", " email ", text)
    text = re.sub(r" e\-mail ", " email ", text)
        
punctuation='["\'?,\.]' # I will replace all these punctuation with ''
abbr_dict={
    "what's":"what is",
    "what're":"what are",
    "who's":"who is",
    "who're":"who are",
    "where's":"where is",
    "where're":"where are",
    "when's":"when is",
    "when're":"when are",
    "how's":"how is",
    "how're":"how are",

    "i'm":"i am",
    "we're":"we are",
    "you're":"you are",
    "they're":"they are",
    "it's":"it is",
    "he's":"he is",
    "she's":"she is",
    "that's":"that is",
    "there's":"there is",
    "there're":"there are",

    "i've":"i have",
    "we've":"we have",
    "you've":"you have",
    "they've":"they have",
    "who've":"who have",
    "would've":"would have",
    "not've":"not have",

    "i'll":"i will",
    "we'll":"we will",
    "you'll":"you will",
    "he'll":"he will",
    "she'll":"she will",
    "it'll":"it will",
    "they'll":"they will",

    "isn't":"is not",
    "wasn't":"was not",
    "aren't":"are not",
    "weren't":"were not",
    "can't":"can not",
    "couldn't":"could not",
    "don't":"do not",
    "didn't":"did not",
    "shouldn't":"should not",
    "wouldn't":"would not",
    "doesn't":"does not",
    "haven't":"have not",
    "hasn't":"has not",
    "hadn't":"had not",
    "won't":"will not",
    punctuation:'',
    '\s+':' ', # replace multi space with one single space
}

In [5]:
# misinfo_train_events.replace(abbr_dict,regex=True,inplace=True)
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
misinfo_train_events_new = []
for event in misinfo_train_events:
    event_re = event.replace("\n"," ")
    tokenize_words = event_re.split()
    pdb.set_trace()
    rep = [abbr_dict[x] if x in abbr_dict else x for x in tokenize_words]


> <ipython-input-5-17214cb86a31>(9)<module>()
-> rep = [abbr_dict[x] if x in abbr_dict else x for x in tokenize_words]
(Pdb) n
> <ipython-input-5-17214cb86a31>(5)<module>()
-> for event in misinfo_train_events:
(Pdb) rep
['why', 'houston', 'flooding', 'isn‘t', 'a', 'sign', 'of', 'climate', 'change', 'Distinguished', 'US', 'climate', 'scientist,', 'Dr', 'Roy', 'Spencer', 'writes:', '"In', 'the', 'context', 'of', 'climate', 'change,', 'is', 'what', 'we', 'are', 'seeing', 'in', 'Houston', 'a', 'new', 'level', 'of', 'disaster', 'which', 'is', 'becoming', 'more', 'common?', 'The', 'flood', 'disaster', 'unfolding', 'in', 'Houston', 'is', 'certainly', 'very', 'unusual.', 'But', 'so', 'are', 'other', 'natural', 'weather', 'disasters,', 'which', 'have', 'always', 'occurred', 'and', 'always', 'will', 'occur....Major', 'floods', 'are', 'difficult', 'to', 'compare', 'throughout', 'history', 'because', 'of', 'the', 'ways', 'we', 'alter', 'the', 'landscape.,', 'For', 'example,', 'as', 'cities', 'lik

(Pdb) quit


BdbQuit: 

In [3]:
# print(misinfo_train_events[0])

In [4]:
# def print_plot(index):
#     example = df[df.index == index][['post', 'tags']].values[0]
#     if len(example) > 0:
#         print(example[0])
#         print('Tag:', example[1])
# print_plot(10)

In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
#     text = BeautifulSoup(text, "lxml").text # HTML decoding
#     pdb.set_trace()
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
    
# df['post'] = df['post'].apply(clean_text)
# print_plot(10)

In [15]:
print('just' in STOPWORDS)
print(STOPWORDS)

True
{'themselves', "should've", 'is', 'same', 'been', 'haven', 'with', 'shouldn', 'ain', 'most', 'myself', 'for', 'than', 'we', "don't", "isn't", 'him', 'he', 'as', 'few', 'where', "you're", 'my', 're', "it's", 'was', "you'd", "hasn't", 'hasn', 'their', 'doing', 'this', 'by', "wasn't", 'm', 'against', 'each', "mightn't", 'won', 'only', 'any', 'will', 'down', 'why', 'were', 'over', 'a', 't', 'theirs', 'yours', 'under', 'needn', "mustn't", "doesn't", 'own', "weren't", 'to', 'don', 'further', 'there', 'mustn', 'more', 'these', 'or', 'other', 'herself', 'having', 'from', "won't", 'those', 'shan', 'll', 'before', 'ma', 'whom', 'hadn', 'and', 'of', "needn't", 'his', 'what', 'not', "you'll", 'between', 'just', 'who', "wouldn't", 'the', 'isn', 'in', 'it', 'her', "shan't", 'nor', 'y', 'o', 'should', 'wasn', 'if', 'out', 've', 'them', "that'll", 'are', 'weren', 'ours', 'then', 'during', 'no', 'mightn', 'does', 'but', 'you', 'its', 'which', 's', 'all', 'itself', "shouldn't", 'wouldn', 'himself',

In [6]:
for i,event in enumerate(misinfo_train_events):
    misinfo_train_events[i] = clean_text(misinfo_train_events[i])
for i,event in enumerate(dev_events):
    dev_events[i] = clean_text(dev_events[i])
for i,event in enumerate(test_unlabel):
    test_unlabel[i] = clean_text(test_unlabel[i])
# print(misinfo_train_events[0])
# print(dev_events[0])
# print(test_unlabel[0])

In [6]:
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

In [7]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [8]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
# train, test = train_test_split(df, test_size=0.3, random_state = 42)

train_tokenized = []
dev_tokenized = []
test_tokenized = []
for event in misinfo_train_events:
    train_tokenized.append(w2v_tokenize_text(event))
for event in dev_events:
    dev_tokenized.append(w2v_tokenize_text(event))
for event in test_unlabel:
    test_tokenized.append(w2v_tokenize_text(event))  
# test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['post']), axis=1).values
# train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['post']), axis=1).values


In [ ]:
X_train_word_average = word_averaging_list(wv,train_tokenized)
X_dev_word_average = word_averaging_list(wv,dev_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

In [11]:
# print(test_tokenized[:2])

In [26]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

def evaluate_result(groundtruth, prediction):
    prediction = prediction.tolist()
    for i,item in enumerate(prediction):
        if item == -1:
            prediction[i] = 0
    p, r, f, _ = precision_recall_fscore_support(groundtruth, prediction, pos_label=1, average="binary")
#     pdb.set_trace()
    print("Performance on the positive class (documents with misinformation):")
    print("Precision =", p)
    print("Recall    =", r)
    print("F1        =", f)
    print(accuracy_score(dev_labels,prediction))
    
def transfer_output(preds):
    test_dic = {}
    for i,test_pre in enumerate(preds):
        if test_pre == -1:
            dic = {}
            dic['label'] = 0
    #         pdb.set_trace()
            test_dic['test-%s'%(i)] = dic
        else:
            dic = {}
    #         pdb.set_trace()
            dic['label'] = 1
            test_dic['test-%s'%(i)] = dic
    jstr = json.dumps(test_dic,ensure_ascii = False)
    # print(jstr)
    with open('test-output.json','w') as f:
        f.write(jstr)

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm

nus = [0.001, 0.01, 0.1, 1]
gammas = [0.001, 0.01, 0.1, 1]
kernels = ['rbf','linear']
for kernel in kernels:
    for nu in nus:
        for gamma in gammas:
            ocs = svm.OneClassSVM(gamma=gamma,kernel = kernel, nu = nu).fit(X_train_word_average)
            y_pre = ocs.predict(X_dev_word_average)
            print(ocs)
            evaluate_result(dev_labels,y_pre)
# logreg = LogisticRegression(n_jobs=1, C=1e5)
# logreg = logreg.fit(X_train_word_average, misinfo_train_labels)
# # y_pred = logreg.predict(X_dev_word_average)
# print('accuracy %s' % accuracy_score(y_pred, test.tags))
# print(classification_report(test.tags, y_pred,target_names=my_tags))

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5102040816326531
Recall    = 1.0
F1        = 0.6756756756756758
0.52
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5102040816326531
Recall    = 1.0
F1        = 0.6756756756756758
0.52
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, to

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='rbf',
            max_iter=-1, nu=0.1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.6125
Recall    = 0.98
F1        = 0.7538461538461539
0.68
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='rbf',
            max_iter=-1, nu=0.1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.6125
Recall    = 0.98
F1        = 0.7538461538461539
0.68
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
            max_iter=-1, nu=0.1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.6125
Recall    = 0.98
F1        = 0.7538461538461539
0.68
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='rbf',
            max_iter=-1, nu=0.1, shrinking=True, tol=0.001, v

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='rbf',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5


...

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
ocs = svm.OneClassSVM(gamma=0.001,kernel = 'rbf', nu = 0.1).fit(X_train_word_average)
test_pres = ocs.predict(X_test_word_average)

print(test_pres)

[1 1 1 ... 1 1 1]


In [52]:
print(sum(test_pres))

92


In [54]:
transfer_output(test_pres)

In [36]:
print(y_pre)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1]


In [37]:
print(dev_labels)

[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1]


In [ ]:
#########################own#########################

In [9]:
from gensim.models import Word2Vec
all_texts = train_tokenized + dev_tokenized + test_tokenized
word_model = Word2Vec(all_texts, min_count=2, size=100, window=8, iter=100, sg = 0)

/usr/local/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [11]:
from gensim.models import Word2Vec
all_texts = train_tokenized + dev_tokenized + test_tokenized
word_model_skip_gram = Word2Vec(all_texts, min_count=2, size=60, window=8, iter=50, sg = 1)

/usr/local/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [33]:
word_model_skip_gram.save("word_model_skip_gram.w2v")

In [14]:
word_model_skip_gram = gensim.models.word2vec.Word2Vec.load("word_model_skip_gram.w2v").wv

In [12]:
import nltk
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


wnl = WordNetLemmatizer()
tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))
punc = punctuation + u'.,;《》？！“”‘’@#￥%…&×（）——+【】{};；●，。&～、|\s:：'
# exclude = set(punc)

def get_wordnet_pos(treebank_tag):

    if (treebank_tag.startswith('J'))|(treebank_tag =='ADJ'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif (treebank_tag.startswith('R'))|(treebank_tag=='ADV'):
        return wordnet.ADV
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    else:
        return wordnet.NOUN

def preprocess_drop_punc_evnets(events):
#     preprocess_drop_punc_evnets = events
    preprocess_drop_punc_evnets = []
#     pdb.set_trace()
    for event in events:
        event_re = re.sub(r"[{}]+".format(punc)," ",event)
        preprocess_drop_punc_evnets.append(event_re)
    return preprocess_drop_punc_evnets

def preprocess_stopwords_lemma_evnets(events):
    preprocess_BOW_events = []
    preprocess_events = []
    preprocess_events_tokens = []
#     pdb.set_trace()
    after_drop_punc_evnets = preprocess_drop_punc_evnets(events)
    for event in after_drop_punc_evnets:
        dic_bow = defaultdict(int)
        tokenize_words = tt.tokenize(event)
#         rep = [abbr_dict[x] if x in abbr_dict else x for x in tokenize_words]
        words_pos = nltk.pos_tag(tokenize_words, tagset="universal")
#         pdb.set_trace()
        filter_event = [(w,pos) for (w,pos) in words_pos if w.lower() not in stopwords]
        lemma_event = []
        for (word,pos) in filter_event:
#             pdb.set_trace()
            word_lemma = wnl.lemmatize(word.lower(),get_wordnet_pos(pos))
            lemma_event.append(word_lemma)
            dic_bow[word_lemma] += 1
        preprocess_events_tokens.append(lemma_event,)
        preprocess_events.append(' '.join(lemma_event))
        preprocess_BOW_events.append(dic_bow)
    return preprocess_events_tokens,preprocess_events,preprocess_BOW_events

preprocess_train_events_tokens,preprocessed_train_events,preprocessed_train_BOW_events = preprocess_stopwords_lemma_evnets(misinfo_train_events)
preprocess_dev_events_tokens,preprocessed_dev_events,preprocessed_dev_BOW_events = preprocess_stopwords_lemma_evnets(dev_events)
preprocess_test_events_tokens,preprocessed_test,preprocessed_test_BOW = preprocess_stopwords_lemma_evnets(test_unlabel)

In [13]:
print(len(preprocess_train_events_tokens))

1168


In [16]:
class MeanEmbeddingVectorizer(object):

    def __init__(self, word_model):
        self.word_model = word_model
        self.vector_size = word_model.wv.vector_size

    def fit(self):  # comply with scikit-learn transformer requirement
        return self

    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector

    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.


        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """
        mean = []
        for word in sent:
            if word in self.word_model.wv.vocab:
                mean.append(self.word_model.wv.get_vector(word))

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.

        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])

In [17]:
from collections import defaultdict
class TfidfEmbeddingVectorizer(object):

    def __init__(self, word_model):

        self.word_model = word_model
        self.word_idf_weight = None
        self.vector_size = word_model.wv.vector_size

    def fit(self, docs):  # comply with scikit-learn transformer requirement
        """
        Fit in a list of docs, which had been preprocessed and tokenized,
        such as word bi-grammed, stop-words removed, lemmatized, part of speech filtered.
        Then build up a tfidf model to compute each word's idf as its weight.
        Noted that tf weight is already involved when constructing average word vectors, and thus omitted.
        :param
            pre_processed_docs: list of docs, which are tokenized
        :return:
            self
        """

        text_docs = []
        for doc in docs:
            text_docs.append(" ".join(doc))

        tfidf = TfidfVectorizer()
        tfidf.fit(text_docs)  # must be list of text string

        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)  # used as default value for defaultdict
        self.word_idf_weight = defaultdict(lambda: max_idf,[(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()])
        return self


    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector


    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """

        mean = []
        for word in sent:
            if word in self.word_model.wv.vocab:
                mean.append(self.word_model.wv.get_vector(word) * self.word_idf_weight[word])  # idf weighted

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])

In [32]:
mean_vec_tr = MeanEmbeddingVectorizer(word_model)
train_vec = mean_vec_tr.transform(train_tokenized)
dev_vec = mean_vec_tr.transform(dev_tokenized)
test_vec = mean_vec_tr.transform(test_tokenized)

NameError: name 'word_model' is not defined

In [18]:
mean_vec_tr_skip_gram = MeanEmbeddingVectorizer(word_model_skip_gram)
train_vec_skip_gram = mean_vec_tr_skip_gram.transform(preprocess_train_events_tokens)
dev_vec_skip_gram = mean_vec_tr_skip_gram.transform(preprocess_dev_events_tokens)
test_vec_skip_gram = mean_vec_tr_skip_gram.transform(preprocess_test_events_tokens)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm

nus = [0.001, 0.01, 0.1, 1]
gammas = [0.001, 0.01, 0.1, 1]
kernels = ['rbf','linear']
for kernel in kernels:
    for nu in nus:
        for gamma in gammas:
            ocs = svm.OneClassSVM(gamma=gamma,kernel = kernel, nu = nu).fit(train_vec_skip_gram)
            y_pre = ocs.predict(dev_vec_skip_gram)
            print(ocs)
            evaluate_result(dev_labels,y_pre)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5050505050505051
Recall    = 1.0
F1        = 0.6711409395973155
0.51
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5050505050505051
Recall    = 1.0
F1        = 0.6711409395973155
0.51
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5050505050505051
Recall    = 1.0
F1        = 0.6711409395973155
0.51
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='rbf',
            max_iter=-

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='rbf',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='linear',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5384615384615384
Recall    = 0.98
F1        = 0.6950354609929078
0.57
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='linear',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performa

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='linear',
            max_iter=-1, nu=0.1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.6333333333333333
Recall    = 0.76
F1        = 0.6909090909090909
0.66
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
ocs = svm.OneClassSVM(gamma=1,kernel = 'linear', nu = 0.1).fit(train_vec_skip_gram)
test_pres = ocs.predict(test_vec_skip_gram)

print(test_pres)

[ 1 -1 -1 ...  1  1 -1]


In [20]:
print(sum(test_pres))

-556


In [23]:
transfer_output(test_pres)

In [29]:
tfidf_vec_tr = TfidfEmbeddingVectorizer(word_model)
tfidf_vec_tr.fit(train_tokenized)  # fit tfidf model first
tfidf_train_vec = tfidf_vec_tr.transform(train_tokenized)
tfidf_dev_vec = tfidf_vec_tr.transform(dev_tokenized)
tfidf_test_vec = tfidf_vec_tr.transform(test_tokenized)

In [21]:
tfidf_vec_tr_skip_gram = TfidfEmbeddingVectorizer(word_model_skip_gram)
tfidf_vec_tr_skip_gram.fit(preprocess_train_events_tokens)  # fit tfidf model first
tfidf_train_vec_skip_gram = tfidf_vec_tr_skip_gram.transform(preprocess_train_events_tokens)
tfidf_dev_vec_skip_gram = tfidf_vec_tr_skip_gram.transform(preprocess_dev_events_tokens)
tfidf_test_vec_skip_gram = tfidf_vec_tr_skip_gram.transform(preprocess_test_events_tokens)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:53: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [25]:
nus = [0.001, 0.01, 0.1, 1]
gammas = [0.001, 0.01, 0.1, 1]
kernels = ['rbf','linear']
for kernel in kernels:
    for nu in nus:
        for gamma in gammas:
            ocs = svm.OneClassSVM(gamma=gamma,kernel = kernel, nu = nu).fit(tfidf_train_vec_skip_gram)
            y_pre = ocs.predict(tfidf_dev_vec_skip_gram)
            print(ocs)
            evaluate_result(dev_labels,y_pre)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5102040816326531
Recall    = 1.0
F1        = 0.6756756756756758
0.52
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.532608695652174
Recall    = 0.98
F1        = 0.6901408450704226
0.56
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.6197183098591549
Recall    = 0.88
F1        = 0.7272727272727273
0.67
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='rbf',
            max_iter=

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='rbf',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='linear',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5
Recall    = 1.0
F1        = 0.6666666666666666
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='linear',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positi

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Performance on the positive class (documents with misinformation):
Precision = 0.4897959183673469
Recall    = 0.96
F1        = 0.6486486486486486
0.48
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='linear',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
ocs = svm.OneClassSVM(gamma=0.1,kernel = 'rbf', nu = 0.01).fit(tfidf_train_vec_skip_gram)
test_pres = ocs.predict(tfidf_test_vec_skip_gram)

print(test_pres)

[-1 -1 -1 ...  1  1 -1]


In [27]:
print(sum(test_pres))

-674


In [29]:
transfer_output(test_pres)

In [30]:
ocs = svm.OneClassSVM(gamma=0.1,kernel = 'rbf', nu = 0.1).fit(tfidf_train_vec_skip_gram)
test_pres = ocs.predict(tfidf_test_vec_skip_gram)

print(test_pres)

[-1 -1 -1 ...  1  1 -1]


In [31]:
print(sum(test_pres))

-674


In [20]:
from sklearn.neighbors import LocalOutlierFactor
n_neighborss = [10,20,30,40,100]
contaminations = [0.1,0.3,0.5]
for n_neighbor in n_neighborss:
    for contamination in contaminations:
        clf = LocalOutlierFactor(n_neighbors = n_neighbor, novelty=True, contamination=contamination)
        clf.fit(train_vec_skip_gram)
        y_pre = clf.predict(dev_vec_skip_gram)
        print(clf)
        evaluate_result(dev_labels,y_pre)

LocalOutlierFactor(algorithm='auto', contamination=0.1, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.5853658536585366
Recall    = 0.96
F1        = 0.7272727272727272
0.64
LocalOutlierFactor(algorithm='auto', contamination=0.3, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.640625
Recall    = 0.82
F1        = 0.7192982456140351
0.68
LocalOutlierFactor(algorithm='auto', contamination=0.5, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.7619047619047619
Recall    = 0.64
F1   

In [22]:
from sklearn.neighbors import LocalOutlierFactor
n_neighborss = [10,20,30,40,100]
contaminations = [0.1,0.3,0.5]
for n_neighbor in n_neighborss:
    for contamination in contaminations:
        clf = LocalOutlierFactor(n_neighbors = n_neighbor, novelty=True, contamination=contamination)
        clf.fit(tfidf_train_vec_skip_gram)
        y_pre = clf.predict(tfidf_dev_vec_skip_gram)
        print(clf)
        evaluate_result(dev_labels,y_pre)

LocalOutlierFactor(algorithm='auto', contamination=0.1, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.5897435897435898
Recall    = 0.92
F1        = 0.71875
0.64
LocalOutlierFactor(algorithm='auto', contamination=0.3, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.6851851851851852
Recall    = 0.74
F1        = 0.7115384615384615
0.7
LocalOutlierFactor(algorithm='auto', contamination=0.5, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.8
Recall    = 0.56
F1        = 0.65882352

In [23]:
clf = LocalOutlierFactor(n_neighbors = 40, novelty=True, contamination=0.1)
clf.fit(tfidf_train_vec_skip_gram)
test_pres = clf.predict(tfidf_test_vec_skip_gram)
print(test_pres)

[ 1 -1  1 ...  1  1 -1]


In [27]:
transfer_output(test_pres)